In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
from multiprocess import Pool
import update_idf_v1 as update_idf
import sys
import os 
import glob
import numpy as np
sys.platform

'win32'

# Main Loop

In [4]:
if __name__ ==  '__main__': 
    building_groups = ['pre1980', 'post1980', 'new']
    climate_zones = ['3b']#['1a','2a','2b','3a','3b','3bc','3c','4a','4b','4c','5a','5b','6a','6b','7a','8a']
    run_updater = False
    all_result = []
    for bldg_group in building_groups:
        input_tuples = list(zip([bldg_group]*len(climate_zones) ,climate_zones, [run_updater]*len(climate_zones)))
        num_processors = len(climate_zones)
        with Pool(processes = num_processors) as p: 
            results = p.map_async(update_idf.run_cz_conversion, input_tuples)
            p.close()
            p.join()
            all_result.append(results)


# Patch Missing

In [53]:
project_root = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings"

pre1980_3b_dir = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\pre1980\idf_files\v7_2\refbldg_3b_usa_nv_las_vegas_pre1980_v1-4_7-2"
post1980_3b_dir = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\post1980\idf_files\v7_2\refbldg_3b_usa_nv_las_vegas_post1980_v1-4_7-2"
new_3b_dir = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\new\idf_files\v7_2\refbldg_3b_usa_nv_las_vegas_new2004_v1-4_7-2"

pre1980_7_2 = glob.glob(os.path.join(pre1980_3b_dir,'*.idf'))
post1980_7_2 = glob.glob(os.path.join(post1980_3b_dir,'*.idf'))
new_7_2 = glob.glob(os.path.join(new_3b_dir,'*.idf'))
idf_patch_list = [pre1980_7_2, post1980_7_2, new_7_2]
bldg_group_patch_list = ['pre1980','post1980','new']

cz = '3b'
run_bool = True
patch_inputs = []

for n, bldg_group in enumerate(bldg_group_patch_list):
    bldg_group_dir = os.path.join(project_root, 'eplus_simulations', bldg_group)
    temp_dir = os.path.join(bldg_group_dir,'idf_files','v7_2', f'temp_{cz}')
    dest_dir = os.path.join(bldg_group_dir,'idf_files','v22_2',f'{cz}_{bldg_group}_v22_2')
    if len(glob.glob(os.path.join(dest_dir,"*")))>0:
        existing_idfs = glob.glob(os.path.join(dest_dir,"*"))
    for idf in idf_patch_list[n]:
        bldg_type = idf.split(os.sep)[-1].lower().split(bldg_group)[0]
        exist_sum_check = [1 for ex in existing_idfs if bldg_type in ex.lower()]
        if sum(exist_sum_check)>0:
            pass
        else:
            input_tuple_patch = (run_bool, idf, temp_dir, dest_dir)
            patch_inputs.append(input_tuple_patch)
            
print(len(patch_inputs))
        

2


In [54]:
if __name__ ==  '__main__': 
    num_processors = 2
    
    chunks = update_idf.chunk_into_n(patch_inputs, num_processors)
    
    with Pool(processes = num_processors) as p: 
        results = p.map_async(update_idf.run_cz_conversion_patch, chunks)
        p.close()
        p.join()


In [51]:
chunks

[[(True,
   'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\pre1980\\idf_files\\v7_2\\refbldg_3b_usa_nv_las_vegas_pre1980_v1-4_7-2\\RefBldgHospitalPre1980_v1.4_7.2_3B_USA_NV_LAS_VEGAS.idf',
   'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\pre1980\\idf_files\\v7_2\\temp_3b',
   'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\pre1980\\idf_files\\v22_2\\3b_pre1980_v22_2')],
 [(True,
   'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\pre1980\\idf_files\\v7_2\\refbldg_3b_usa_nv_las_vegas_pre1980_v1-4_7-2\\RefBldgPrimarySchoolPre1980_v1.4_7.2_3B_USA_NV_LAS_VEGAS.idf',
   'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\pre1980\\idf_files\\v7_2\\temp_3b',
   'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\pre1980\\idf_files\\v22_2\\3b_pre1980_v22_2')],
 [(True,
   'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_

In [41]:
chunks[0]

[(True,
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\pre1980\\idf_files\\v7_2\\refbldg_3b_usa_nv_las_vegas_pre1980_v1-4_7-2\\RefBldgFullServiceRestaurantPre1980_v1.4_7.2_3B_USA_NV_LAS_VEGAS.idf',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\pre1980\\idf_files\\v7_2\\temp_3b',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\pre1980\\idf_files\\v22_2\\3b_pre1980_v22_2'),
 (True,
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\post1980\\idf_files\\v7_2\\refbldg_3b_usa_nv_las_vegas_post1980_v1-4_7-2\\RefBldgMidriseApartmentPost1980_v1.4_7.2_3B_USA_NV_LAS_VEGAS.idf',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\post1980\\idf_files\\v7_2\\temp_3b',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\post1980\\idf_files\\v22_2\\3b_post1980_v22_2'),
 (True,
  'C:\\Users\\Justin\\Documents\\GitH